### Scoring using the best model locally - Optional

The training environment in AML compute cluster can be different than the one locally. We need to create a new Conda environment in the repo so that model can be loaded locally. 

    conda env create -f conda_env.yml

Use the model file downloaded using automl_amlsdkv2_training.ipynb. 

If AML Compute Instance is used for local scoring, then the notebook kernel should be set to Python 3.8 - AzureML

If AML Compute Cluster is used for remote scoring, then the notebook kernel should be set to Python 3.10 - SDK v2

### Score using Compute Cluster in AML - Optional 

In [1]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

#### Provide values for your AML workspace

    subscription_id = "YOUR_SUBSCRIPTION_ID"
    resource_group = "YOUR_RESOURCE_GROUP"
    workspace = "YOUR_WORKSPACE_NAME"

In [2]:

ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your Azure Machine Learning workspace
    subscription_id = "f1a8fafd-a8a3-46d8-bb5e-01deb63d275d"
    resource_group = "aml-rg"
    workspace = "aml-testws"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)

We could not find config.json in: . or in its parent directories. Please provide the full path to the config file or ensure that config.json exists in the parent directories.


Set the model name created in automl_amlsdkv2_training.ipynb

In [4]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import ModelType

my_unique_dataset_name = "1155c2a891714097b099ade1fee4bddb" # Replace with your dataset name used in the training notebook

model_name = f"{my_unique_dataset_name}-wa_telco_customer_churn_model_best"
model_versions = ml_client.models.list(name=model_name)
latest_model = max(model_versions, key=lambda x: x.version)
latest_model.path

'azureml://subscriptions/f1a8fafd-a8a3-46d8-bb5e-01deb63d275d/resourceGroups/aml-rg/workspaces/aml-testws/datastores/workspaceartifactstore/paths/ExperimentRun/dcid.jolly_circle_ywmvvvb4dc_3/outputs/mlflow-model'

In [7]:
from azure.ai.ml import command, Input, Output
from azure.ai.ml.constants import AssetTypes, InputOutputModes
data_type = AssetTypes.MLTABLE
input_mode = InputOutputModes.RO_MOUNT
output_mode = InputOutputModes.RW_MOUNT

Get reference to test dataset from AML

In [5]:

data_asset_name = f"{my_unique_dataset_name}-wa_telco_customer_churn_test_data"
data_asset_version = "1.0" # Replace with your dataset version used in the training notebook
data_asset = ml_client.data.get(name=data_asset_name, version=data_asset_version)

Create Input schema for scoring job

In [8]:

#data_path = "./wa_telco_customer_churn_test_data_uri_file.csv"
inputs = {
    "input_data": Input(type=AssetTypes.URI_FILE, path=data_asset.path),
    "input_model": Input(type=AssetTypes.MLFLOW_MODEL, path=latest_model.path)
}

In [ ]:
inputs

Create Output schema for scoring job.
Update dataset_version if the scoring job is re-run. 

In [13]:
dataset_path = f"{my_unique_dataset_name}-wa-telco-customer-churn-predictions"
dataset_version = "2.0" # provide a new version number for the predictions dataset if scoring is run multiple times
outputs = {
    "output_folder": Output(
        type=AssetTypes.URI_FOLDER,
        path=f"azureml://subscriptions/{ml_client.subscription_id}/resourcegroups/{ml_client.resource_group_name}/workspaces/{ml_client.workspace_name}/datastores/workspaceblobstore/paths/{dataset_path}",
        name = dataset_path,
        version = dataset_version
    )
}

Define scoring job using Command component

Use the same environment as the model was created in AutoML. 
    
        AzureML-AutoML:142

Please review score.py

In [14]:
from azure.ai.ml import command, Input


# define the command
command_job = command(
    code=".././score-component/score_src/",
    command="python score.py --input_model ${{inputs.input_model}} --input_data ${{inputs.input_data}} --output_folder ${{outputs.output_folder}}",
    experiment_name=f"{my_unique_dataset_name}-wa-telco-customer-churn-prediction",
    inputs=inputs,
    outputs=outputs,    
    environment="AzureML-AutoML:142",
    compute="cpu-cluster",
)

# Submit the command
ml_client.jobs.create_or_update(command_job)

Experiment,Name,Type,Status,Details Page
1155c2a891714097b099ade1fee4bddb-wa-telco-customer-churn-prediction,maroon_rail_qcx1lvfyj7,command,Starting,Link to Azure Machine Learning studio


Please review the results of the Scoring job in Aml Workspace. 

- Once the Job complete, to view the Scoring results, naviate to the Azure Portal from the AML workspace top right corner. 
![Alt text](.././assets/automlimage41.png)

- Click on "View all properties in Azure Portal".  This will open the Azure Poral in a new browser tab. 
- In the Azure portal, click on the Storage link in the AML workspace overview panel. 
![Alt text](.././assets/automlimage42.png)
![Alt text](.././assets/automlimage43.png)

- Navigate to Containers tab.

![Alt text](.././assets/automlimage44.png)

- Navigate to the azureml-blobstore-{{some-guid}} folder. Type the folder path in the search box. For e.g azureml/my_unique_folder_name/. This is the name of the folder path provided in the Designer page. Locate the my_unique_dataset_name-wa-telco-customer-churn-predictions folder created in this notebook.

![Alt text](.././assets/automlimage45.png)

- Download the file to review the Churn Prediction Results. 

![Alt text](.././assets/automlimage46.png)